<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Practice SQL with Pandas pt. 2

---


We've learned about relational databases and the language most used to query them, SQL.  

In this lab we are going to gain more practice converting information to a SQL DB, querying the data and then analysing it with Python.

In [1]:
# Necessary Libraries
import pandas as pd
import sqlite3
import numpy as np

### 1.  Read in the EuroMart CSV Data
- 'EuroMart-ListOfOrders.csv'
- 'EuroMart-OrderBreakdown.csv'
- 'EuroMart-SalesTargets.csv'

In [2]:
# Reading CSV to Dataframe
orders = pd.read_csv('./datasets/EuroMart-ListOfOrders.csv', encoding = 'utf-8')
OBD =  pd.read_csv('./datasets/EuroMart-OrderBreakdown.csv', encoding = 'utf-8')
sales_targets =  pd.read_csv('./datasets/EuroMart-SalesTargets.csv', encoding = 'utf-8')

### 2. Rename columns to remove any spaces

In [3]:
name_list = ["order_id","order_date","customer_name","city","country","region","segment","ship_date","ship_mode","state"]

In [4]:
name_list1 = ["order_id","product_name","discount","sales","profit","quantity","category","sub_category"]

In [5]:
name_list2 = ["month_order_date","category","target"]

In [6]:
orders.columns=name_list

In [7]:
OBD.columns=name_list1

In [8]:
sales_targets.columns=name_list2

In [9]:
orders.head()

,order_id,order_date,customer_name,city,country,region,segment,ship_date,ship_mode,state
0,BN-2011-7407039,1/1/2011,Ruby Patel,Stockholm,Sweden,North,Home Office,1/5/2011,Economy Plus,Stockholm
1,AZ-2011-9050313,1/3/2011,Summer Hayward,Southport,United Kingdom,North,Consumer,1/7/2011,Economy,England
2,AZ-2011-6674300,1/4/2011,Devin Huddleston,Valence,France,Central,Consumer,1/8/2011,Economy,Auvergne-Rhône-Alpes
3,BN-2011-2819714,1/4/2011,Mary Parker,Birmingham,United Kingdom,North,Corporate,1/9/2011,Economy,England
4,AZ-2011-617423,1/5/2011,Daniel Burke,Echirolles,France,Central,Home Office,1/7/2011,Priority,Auvergne-Rhône-Alpes


In [10]:
OBD.head()

,order_id,product_name,discount,sales,profit,quantity,category,sub_category
0,BN-2011-7407039,"Enermax Note Cards, Premium",0.5,$45.00,-$26.00,3,Office Supplies,Paper
1,AZ-2011-9050313,"Dania Corner Shelving, Traditional",0.0,$854.00,$290.00,7,Furniture,Bookcases
2,AZ-2011-6674300,"Binney & Smith Sketch Pad, Easy-Erase",0.0,$140.00,$21.00,3,Office Supplies,Art
3,BN-2011-2819714,"Boston Markers, Easy-Erase",0.5,$27.00,-$22.00,2,Office Supplies,Art
4,BN-2011-2819714,"Eldon Folders, Single Width",0.5,$17.00,-$1.00,2,Office Supplies,Storage


In [11]:
sales_targets.head()

,month_order_date,category,target
0,Jan-11,Furniture,"$10,000.00"
1,Feb-11,Furniture,"$10,100.00"
2,Mar-11,Furniture,"$10,300.00"
3,Apr-11,Furniture,"$10,400.00"
4,May-11,Furniture,"$10,500.00"


### 3. Remove dollar signs from sales and profit columns in the order breakdown dataframe.

> **Note:** Remember to convert the columns to float as well. 

In [12]:
OBD["sales"] = OBD["sales"].str.replace("$","")

In [13]:
OBD["sales"] = OBD["sales"].str.replace(",","")

In [14]:
OBD["profit"] = OBD["profit"].str.replace("$","")

In [15]:
OBD["profit"] = OBD["profit"].str.replace(",","")

In [16]:
OBD["sales"] = OBD["sales"].map(lambda x:float(x))

In [17]:
OBD["profit"] = OBD["profit"].map(lambda x:float(x))

In [18]:
OBD.dtypes

order_id         object
product_name     object
discount        float64
sales           float64
profit          float64
quantity          int64
category         object
sub_category     object
dtype: object

### 4. Create a SQL Database called 'EuroMart' and save the three dataframes as SQL tables. 

In [19]:
connection = sqlite3.connect('datasets/EuroMart.db.sqlite')

In [20]:
orders.to_sql(name="new_orders",con=connection,if_exists = 'replace', index = False
)

In [21]:
OBD.to_sql(name="new_OBD",con=connection,if_exists = 'replace', index = False
)

In [22]:
sales_targets.to_sql(name="new_sales_targets",con=connection,if_exists = 'replace', index = False
)

In [23]:
orders.head(1)

,order_id,order_date,customer_name,city,country,region,segment,ship_date,ship_mode,state
0,BN-2011-7407039,1/1/2011,Ruby Patel,Stockholm,Sweden,North,Home Office,1/5/2011,Economy Plus,Stockholm


### 5. How many orders has each Customer placed? 

In [24]:
pd.read_sql('SELECT customer_name FROM new_orders', con=connection).head()

,customer_name
0,Ruby Patel
1,Summer Hayward
2,Devin Huddleston
3,Mary Parker
4,Daniel Burke


> *If you're doubting your output check using Pandas*

### 6. Create a Query to return a table of only geographic features from the List of Orders Table.

In [25]:
orders.head(1)

,order_id,order_date,customer_name,city,country,region,segment,ship_date,ship_mode,state
0,BN-2011-7407039,1/1/2011,Ruby Patel,Stockholm,Sweden,North,Home Office,1/5/2011,Economy Plus,Stockholm


In [26]:
geograph = pd.read_sql('SELECT city,country,region FROM new_orders', con=connection)

In [27]:
geograph.head()

,city,country,region
0,Stockholm,Sweden,North
1,Southport,United Kingdom,North
2,Valence,France,Central
3,Birmingham,United Kingdom,North
4,Echirolles,France,Central


### 7. Create a Query to return a table with all of the orders that had a negative profit from the Order Breakdown Table.

In [28]:
OBD.head(1)

,order_id,product_name,discount,sales,profit,quantity,category,sub_category
0,BN-2011-7407039,"Enermax Note Cards, Premium",0.5,45.0,-26.0,3,Office Supplies,Paper


In [29]:
neg_profit = pd.read_sql('SELECT * FROM new_OBD WHERE profit<0',con=connection)

In [30]:
neg_profit.head()

,order_id,product_name,discount,sales,profit,quantity,category,sub_category
0,BN-2011-7407039,"Enermax Note Cards, Premium",0.5,45.0,-26.0,3,Office Supplies,Paper
1,BN-2011-2819714,"Boston Markers, Easy-Erase",0.5,27.0,-22.0,2,Office Supplies,Art
2,BN-2011-2819714,"Eldon Folders, Single Width",0.5,17.0,-1.0,2,Office Supplies,Storage
3,BN-2011-3248724,"Ikea Classic Bookcase, Metal",0.6,987.0,-1012.0,6,Furniture,Bookcases
4,BN-2011-3248724,"Binney & Smith Sketch Pad, Blue",0.5,116.0,-56.0,5,Office Supplies,Art


### 8. Construct a query to return a table with the Customer Name and Product Name.  

> **Note:** This will require a join!


```SQL  
SELECT table1.column_name(s), table2.column_name(s),
FROM table1
INNER JOIN table2
ON table1.column_name = table2.column_name;
```

In [31]:
cus_pro_names = pd.read_sql(
    'SELECT new_orders.customer_name, new_OBD.product_name FROM new_orders INNER JOIN new_OBD ON new_orders.order_id = new_OBD.order_id',con=connection)

In [32]:
cus_pro_names.head()

,customer_name,product_name
0,Ruby Patel,"Enermax Note Cards, Premium"
1,Summer Hayward,"Dania Corner Shelving, Traditional"
2,Devin Huddleston,"Binney & Smith Sketch Pad, Easy-Erase"
3,Mary Parker,"Boston Markers, Easy-Erase"
4,Mary Parker,"Eldon Folders, Single Width"


### 9.  How many orders for "Office Supplies" (Category) has Sweden made?

> **Note:** from this point on you'll probably be combining SQL and Pandas, in that you would use SQL querys to gather the relevant information and use Pandas to analyze it.

In [33]:
OBD.head(1)

,order_id,product_name,discount,sales,profit,quantity,category,sub_category
0,BN-2011-7407039,"Enermax Note Cards, Premium",0.5,45.0,-26.0,3,Office Supplies,Paper


In [34]:
orders.head(1)

,order_id,order_date,customer_name,city,country,region,segment,ship_date,ship_mode,state
0,BN-2011-7407039,1/1/2011,Ruby Patel,Stockholm,Sweden,North,Home Office,1/5/2011,Economy Plus,Stockholm


In [35]:
sweden_supplies = pd.read_sql(
    "SELECT new_orders.order_id,new_orders.country, new_OBD.category "
    "FROM new_orders "
    "LEFT JOIN new_OBD "
    "ON new_orders.order_id=new_OBD.order_id "
    "WHERE new_orders.country='Sweden' AND new_OBD.category='Office Supplies' "
    , con=connection, index_col='country')

In [36]:
len(sweden_supplies)

133

In [37]:
sweden_supplies = pd.read_sql(
    "SELECT COUNT(new_orders.order_id),new_orders.country, new_OBD.category "
    "FROM new_orders "
    "LEFT JOIN new_OBD "
    "ON new_orders.order_id=new_OBD.order_id "
    "WHERE new_orders.country='Sweden' AND new_OBD.category='Office Supplies' "
    , con=connection, index_col='country')

In [38]:
sweden_supplies

,COUNT(new_orders.order_id),category
country,,
Sweden,133,Office Supplies


### 10. What was the total sales for products that have been discounted? 

In [39]:
orders.head(1)

,order_id,order_date,customer_name,city,country,region,segment,ship_date,ship_mode,state
0,BN-2011-7407039,1/1/2011,Ruby Patel,Stockholm,Sweden,North,Home Office,1/5/2011,Economy Plus,Stockholm


In [40]:
OBD.head(5)

,order_id,product_name,discount,sales,profit,quantity,category,sub_category
0,BN-2011-7407039,"Enermax Note Cards, Premium",0.5,45.0,-26.0,3,Office Supplies,Paper
1,AZ-2011-9050313,"Dania Corner Shelving, Traditional",0.0,854.0,290.0,7,Furniture,Bookcases
2,AZ-2011-6674300,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,21.0,3,Office Supplies,Art
3,BN-2011-2819714,"Boston Markers, Easy-Erase",0.5,27.0,-22.0,2,Office Supplies,Art
4,BN-2011-2819714,"Eldon Folders, Single Width",0.5,17.0,-1.0,2,Office Supplies,Storage


In [41]:
sales_targets.head(1)

,month_order_date,category,target
0,Jan-11,Furniture,"$10,000.00"


In [42]:
total_sales = pd.read_sql(
"SELECT SUM(sales) as total_sales "
    "FROM new_OBD "
    "WHERE 0<discount "
    ,con=connection,index_col='total_sales'
)

In [43]:
total_sales

""
total_sales
1115614.0


In [44]:
total_sales.sum()

Series([], dtype: float64)

### 11. What is the total quantity of objects sold for each country?

In [45]:
orders.head(1)

,order_id,order_date,customer_name,city,country,region,segment,ship_date,ship_mode,state
0,BN-2011-7407039,1/1/2011,Ruby Patel,Stockholm,Sweden,North,Home Office,1/5/2011,Economy Plus,Stockholm


In [46]:
OBD.head(5)

,order_id,product_name,discount,sales,profit,quantity,category,sub_category
0,BN-2011-7407039,"Enermax Note Cards, Premium",0.5,45.0,-26.0,3,Office Supplies,Paper
1,AZ-2011-9050313,"Dania Corner Shelving, Traditional",0.0,854.0,290.0,7,Furniture,Bookcases
2,AZ-2011-6674300,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,21.0,3,Office Supplies,Art
3,BN-2011-2819714,"Boston Markers, Easy-Erase",0.5,27.0,-22.0,2,Office Supplies,Art
4,BN-2011-2819714,"Eldon Folders, Single Width",0.5,17.0,-1.0,2,Office Supplies,Storage


In [47]:
order_counts = pd.read_sql(
                'SELECT order_breakdown.quantity, orders.country '
                'FROM orders '
                'INNER JOIN order_breakdown '
                'ON orders.order_id = order_breakdown.order_id ',
                con = db_connection)

order_counts.groupby('country').sum()

NameError: name 'db_connection' is not defined

In [ ]:
order_counts_grouped = pd.read_sql(
                'SELECT sum(order_breakdown.quantity) as total_quantity, orders.country '
                'FROM orders '
                'INNER JOIN order_breakdown '
                'ON orders.order_id = order_breakdown.order_id '
                'GROUP BY orders.country '
                'ORDER BY total_quantity DESC',
                con = db_connection,index_col='country')

order_counts_grouped

In [ ]:
pd.read_sql(
'SELECT a.country, SUM(b.quantity) as total_sum '
    'FROM new_orders a '
    'INNER JOIN new_OBD b '
    'ON a.order_id = b.order_id '
    'GROUP BY a.country  '
    'ORDER BY b.quantity DESC'
    , con=connection,
)

### 12. In what Countries are profits lowest? (Report lowest 5-10)

In [ ]:
orders.head(2)

In [ ]:
OBD.head(2)

In [ ]:
country_profit = pd.read_sql(
'SELECT a.country, SUM(b.profit) as total_profit '
    'FROM new_orders a '
    'INNER JOIN new_OBD b '
    'ON a.order_id = b.order_id '
    'GROUP BY a.country  '
    'ORDER BY total_profit ASC'
    , con=connection, index_col="country"
)
country_profit.iloc[:6,:]

### 13. What Countries have the best and worst profit to sales ratios?

Total profits divided by total sales -
this is saying for every dollar of product sold, how much is the profit.

In [ ]:
orders.head(50)

In [ ]:
OBD.head(40)

In [ ]:
spr = pd.read_sql('SELECT orders.country,'
                   'sum(order_breakdown.profit) as total_profit, '
                   'sum(order_breakdown.sales) as total_sales, '
                   '(sum(order_breakdown.profit) / sum(order_breakdown.sales)) as ratio '
                   'FROM orders '
                   'INNER JOIN order_breakdown '
                   'ON orders.order_id = order_breakdown.order_id '
                   'GROUP BY orders.country '
                   'ORDER BY ratio DESC',
            con = db_connection,index_col='country')
spr

In [ ]:
two = pd.read_sql(
'SELECT a.country, '
    'SUM(b.sales) as total_sales, '
    'SUM(b.profit) as total_profit, '
    '(SUM(b.profit)/SUM(b.sales)) as ratio '
    'FROM new_orders a '
    'INNER JOIN new_OBD b '
    'ON a.order_id = b.order_id '
    'GROUP BY a.country '
    'ORDER BY ratio DESC'
    , con=connection, index_col="country"
    
)
two

In [ ]:
one = pd.read_sql(
'SELECT a.country, b.sales, b.profit '
    'FROM new_orders a '
    'INNER JOIN new_OBD b '
    'ON a.order_id = b.order_id '
    , con=connection
    
)
one.head()

In [ ]:
one = one.groupby("country")

In [ ]:
one = one.sum().sort_values(by="profit")

In [ ]:
one["ratio"] = one["profit"]/one["sales"]

In [ ]:
one = one.sort_values(by="ratio")

In [ ]:
one.tail()

### 14. What Shipping method is most common for 'Bookcases' (Sub Category)

### 15 .What city in the Orders table generated the highest net sales?  (List all the cities and countries in descending order by net sales.)

### 16. [BONUS] Create a Column called 'Shipping Delay' on the 'orders' table, which is the difference in days between 'Order Date' and 'Ship Date'.

### 17. [BONUS] Update your Orders table in your Sqlite DB to include the 'Shipping Delay' feature.

> **Hint:** Remember the df.to_sql command we learnt earlier..

### 18. [BONUS] Which Product Category has the highest average 'Shipping Delay'

## Challenge problem: In what months and Categories were Sales Targets Exceeded?

---

**This problem is more challenging.** 

Try it out if you want to test your skills! Like the bonus questions, it is not expected that you complete it.

This may require a considerable amount of data processing.

**In what months and Categories did Sales fail to exceed their targets?*